In [11]:
# the aim is to make 4 tables of a table called "alltables"

In [12]:
# importing libraries

import mysql.connector
import pandas as pd
from tqdm.notebook import tqdm
import random
from sqlalchemy import create_engine
import numpy as np

# Making the initial table

In [13]:
'''
first we need to read 11 csv files related to province's shops and add a "city" column 
which contains the name of each city then we append all tables in a table called "alltables"
'''

cities = {'arak':'اراک','bandar_abbas':'بندرعباس','gonbad_kavos':'گنبد کاووس','gorgan':'گرگان','hamedan':'همدان','karaj':'کرج','qom':'قم','rasht':'رشت','tehran':'تهران','urumia':'ارومیه','yazd':'یزد'}
def reader(data):
    df = pd.read_csv('{}.csv'.format(data))
    df['city'] = '{}'.format(cities[data])
    return df

dfs =[]
for i in list(cities.keys()):
    dfs.append(reader(i))
alltables = pd.concat(dfs, ignore_index=True)
alltables.drop("Unnamed: 0",axis=1, inplace=True)
alltables = alltables.drop_duplicates(subset=['name','menu','address_by_str'])
alltables.head(3)

,name,rate,delivery,menu,out_of_zone_available,time_available,address_by_str,address_by_geo,city
0,کافه رستوران اسکای,4.8,30 تا 50 دقیقه\nمیانگین زمان ارسال,"[{'name': 'چیز فرایز', 'price': '65,000 تومان'...",دارد (ارسال با آژانس),صبحانه ناهار شام\nشنبه 10:00:00 تا 12:00:00 12...,خیابان عباس آباد، روبه روی پاساژ اسلامی، کوچه ...,"34.08529155124946,49.688402283936746",اراک
1,تهیه غذای منظریه,0.0,30 تا 50 دقیقه\nمیانگین زمان ارسال,"[{'name': 'یک سیخ کوبیده', 'price': '35,000 تو...",دارد (ارسال با آژانس),ناهار شام\nشنبه 11:00:00 تا 15:00:00 18:30:00 ...,اراک ، پایین تر از پل فرنگ، خیابان کاشانی،تهیه...,"34.08920115464067,49.68089746311308",اراک
2,رستوران ژیوان,0.0,20 تا 40 دقیقه\nمیانگین زمان ارسال,"[{'name': 'پیتزا آمریکایی برتولی دو نفره', 'pr...",دارد (ارسال با آژانس),ناهار شام\nشنبه 11:30:00 تا 15:00:00 18:30:00 ...,اراک، خیابان شریعتی، نبش کوچه پامچال,"34.07694749848257,49.69773637130857",اراک


# DRIVING 4 TABLES OUT OF THE INITIAL TABLE
## 1- DRIVING SHOPS TABLE

In [14]:
# splitting the "delivery" column into "min_wait" and "max_wait" columns

alltables['delivery'] = alltables['delivery'].str.rstrip('دقیقه\nمیانگین زمان ارسال')
alltables_sp = alltables['delivery'].str.split('تا', expand=True)
alltables_sp.columns = ['min_wait','max_wait']
alltables['min_wait'] = alltables_sp['min_wait']
alltables['max_wait'] = alltables_sp['max_wait']
alltables.drop("delivery",axis=1, inplace=True)
alltables.head(3)

,name,rate,menu,out_of_zone_available,time_available,address_by_str,address_by_geo,city,min_wait,max_wait
0,کافه رستوران اسکای,4.8,"[{'name': 'چیز فرایز', 'price': '65,000 تومان'...",دارد (ارسال با آژانس),صبحانه ناهار شام\nشنبه 10:00:00 تا 12:00:00 12...,خیابان عباس آباد، روبه روی پاساژ اسلامی، کوچه ...,"34.08529155124946,49.688402283936746",اراک,30,50
1,تهیه غذای منظریه,0.0,"[{'name': 'یک سیخ کوبیده', 'price': '35,000 تو...",دارد (ارسال با آژانس),ناهار شام\nشنبه 11:00:00 تا 15:00:00 18:30:00 ...,اراک ، پایین تر از پل فرنگ، خیابان کاشانی،تهیه...,"34.08920115464067,49.68089746311308",اراک,30,50
2,رستوران ژیوان,0.0,"[{'name': 'پیتزا آمریکایی برتولی دو نفره', 'pr...",دارد (ارسال با آژانس),ناهار شام\nشنبه 11:30:00 تا 15:00:00 18:30:00 ...,اراک، خیابان شریعتی، نبش کوچه پامچال,"34.07694749848257,49.69773637130857",اراک,20,40


In [15]:
# cleaning "address_by_geo" values and separating it into two columns : "lat" and "lon"

alltables_loc = alltables['address_by_geo'].str.split(',', expand=True)
alltables_loc.columns = ['lat','lon']
alltables['lat'] = alltables_loc['lat']
alltables['lon'] = alltables_loc['lon']
alltables.drop("address_by_geo",axis=1, inplace=True)
alltables['lat'] =alltables['lat'].astype(float)
alltables['lon'] =alltables['lon'].astype(float)
alltables['lat']=alltables['lat'].round(decimals = 5)
alltables['lon']=alltables['lon'].round(decimals = 5)
alltables.head(3)

,name,rate,menu,out_of_zone_available,time_available,address_by_str,city,min_wait,max_wait,lat,lon
0,کافه رستوران اسکای,4.8,"[{'name': 'چیز فرایز', 'price': '65,000 تومان'...",دارد (ارسال با آژانس),صبحانه ناهار شام\nشنبه 10:00:00 تا 12:00:00 12...,خیابان عباس آباد، روبه روی پاساژ اسلامی، کوچه ...,اراک,30,50,34.08529,49.68840
1,تهیه غذای منظریه,0.0,"[{'name': 'یک سیخ کوبیده', 'price': '35,000 تو...",دارد (ارسال با آژانس),ناهار شام\nشنبه 11:00:00 تا 15:00:00 18:30:00 ...,اراک ، پایین تر از پل فرنگ، خیابان کاشانی،تهیه...,اراک,30,50,34.08920,49.68090
2,رستوران ژیوان,0.0,"[{'name': 'پیتزا آمریکایی برتولی دو نفره', 'pr...",دارد (ارسال با آژانس),ناهار شام\nشنبه 11:30:00 تا 15:00:00 18:30:00 ...,اراک، خیابان شریعتی، نبش کوچه پامچال,اراک,20,40,34.07695,49.69774


In [16]:
# cleaning "out_of_zone_available" and splitting it into 2 columns: "del_status" and "del_kind"

alltables.loc[:,'out_of_zone_available']
alltables_del = alltables['out_of_zone_available'].str.split('(', expand=True)
alltables_del.columns = ['del_status','del_kind']
alltables_del['del_kind']= alltables_del['del_kind'].str.replace(')', '')
alltables['del_status'] = alltables_del['del_status']
alltables['del_kind'] = alltables_del['del_kind']
alltables.drop("out_of_zone_available",axis=1, inplace=True)
alltables.head(3)

C:\Users\ALPHA\AppData\Local\Temp\ipykernel_6644\1749351379.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  alltables_del['del_kind']= alltables_del['del_kind'].str.replace(')', '')


,name,rate,menu,time_available,address_by_str,city,min_wait,max_wait,lat,lon,del_status,del_kind
0,کافه رستوران اسکای,4.8,"[{'name': 'چیز فرایز', 'price': '65,000 تومان'...",صبحانه ناهار شام\nشنبه 10:00:00 تا 12:00:00 12...,خیابان عباس آباد، روبه روی پاساژ اسلامی، کوچه ...,اراک,30,50,34.08529,49.68840,دارد,ارسال با آژانس
1,تهیه غذای منظریه,0.0,"[{'name': 'یک سیخ کوبیده', 'price': '35,000 تو...",ناهار شام\nشنبه 11:00:00 تا 15:00:00 18:30:00 ...,اراک ، پایین تر از پل فرنگ، خیابان کاشانی،تهیه...,اراک,30,50,34.08920,49.68090,دارد,ارسال با آژانس
2,رستوران ژیوان,0.0,"[{'name': 'پیتزا آمریکایی برتولی دو نفره', 'pr...",ناهار شام\nشنبه 11:30:00 تا 15:00:00 18:30:00 ...,اراک، خیابان شریعتی، نبش کوچه پامچال,اراک,20,40,34.07695,49.69774,دارد,ارسال با آژانس


In [17]:
# adding a unique column for 'name' column which is the name of shops - we previously checked the duplicates

alltables['id'] = [i for i in range(1,alltables.shape[0]+1)]
alltables = alltables.replace([None], [''], regex=True)
alltables['name']= alltables['name'].str.replace('\u200c', '')
alltables['name']= alltables['name'].str.replace('27', 'بیست و هفت')
alltables.head(3)

,name,rate,menu,time_available,address_by_str,city,min_wait,max_wait,lat,lon,del_status,del_kind,id
0,کافه رستوران اسکای,4.8,"[{'name': 'چیز فرایز', 'price': '65,000 تومان'...",صبحانه ناهار شام\nشنبه 10:00:00 تا 12:00:00 12...,خیابان عباس آباد، روبه روی پاساژ اسلامی، کوچه ...,اراک,30,50,34.08529,49.68840,دارد,ارسال با آژانس,1
1,تهیه غذای منظریه,0.0,"[{'name': 'یک سیخ کوبیده', 'price': '35,000 تو...",ناهار شام\nشنبه 11:00:00 تا 15:00:00 18:30:00 ...,اراک ، پایین تر از پل فرنگ، خیابان کاشانی،تهیه...,اراک,30,50,34.08920,49.68090,دارد,ارسال با آژانس,2
2,رستوران ژیوان,0.0,"[{'name': 'پیتزا آمریکایی برتولی دو نفره', 'pr...",ناهار شام\nشنبه 11:30:00 تا 15:00:00 18:30:00 ...,اراک، خیابان شریعتی، نبش کوچه پامچال,اراک,20,40,34.07695,49.69774,دارد,ارسال با آژانس,3


In [18]:
# know it's time to sort and rename the columns and change the type of some columns
alltables = alltables[['id','name','city','rate','min_wait','max_wait','del_status','del_kind','address_by_str','lat','lon','menu','time_available']]
alltables.rename(columns={"lat":"Lat","lon":"Lon","id": "ID","name": "ShopName","city": "City","rate": "Rate","min_wait": "Min_wait","max_wait": "Max_wait","del_status": "Del_status","del_kind": "Del_kind","address_by_str": "Address","menu": "Menu","time_available": "TimeAvail"}, inplace=True)
alltables = alltables.astype({"Min_wait":"int","Max_wait":"int"})


In [19]:
alltables['ShopName'] = alltables['ShopName'].str.strip()
alltables['City'] = alltables['City'].str.strip()
alltables['Del_status'] = alltables['Del_status'].str.strip()
alltables['Del_kind'] = alltables['Del_kind'].str.strip()
alltables['Address'] = alltables['Address'].str.strip()
alltables['Del_status'] = alltables['Del_status'].replace(['دارد'] , '1')
alltables['Del_status'] = alltables['Del_status'].replace(['ندارد'] , '0')
alltables['Del_kind'] = alltables['Del_kind'].replace(['ارسال با آژانس'] , 'آژانس')

In [20]:
#the table looks way better now!
alltables.head(4)

,ID,ShopName,City,Rate,Min_wait,Max_wait,Del_status,Del_kind,Address,Lat,Lon,Menu,TimeAvail
0,1,کافه رستوران اسکای,اراک,4.8,30,50,1,آژانس,خیابان عباس آباد، روبه روی پاساژ اسلامی، کوچه ...,34.08529,49.68840,"[{'name': 'چیز فرایز', 'price': '65,000 تومان'...",صبحانه ناهار شام\nشنبه 10:00:00 تا 12:00:00 12...
1,2,تهیه غذای منظریه,اراک,0.0,30,50,1,آژانس,اراک ، پایین تر از پل فرنگ، خیابان کاشانی،تهیه...,34.08920,49.68090,"[{'name': 'یک سیخ کوبیده', 'price': '35,000 تو...",ناهار شام\nشنبه 11:00:00 تا 15:00:00 18:30:00 ...
2,3,رستوران ژیوان,اراک,0.0,20,40,1,آژانس,اراک، خیابان شریعتی، نبش کوچه پامچال,34.07695,49.69774,"[{'name': 'پیتزا آمریکایی برتولی دو نفره', 'pr...",ناهار شام\nشنبه 11:30:00 تا 15:00:00 18:30:00 ...
3,4,پیتزا تک,اراک,0.0,40,60,1,آژانس,اراک، سه راه ابوذر، خیابان ابوذر، روبروی نانوا...,34.09574,49.68300,"[{'name': 'پیتزا قارچ و پنیر خانواده', 'price'...",ناهار شام\nشنبه 11:00:00 تا 15:30:00 18:30:00 ...


In [21]:
SHOPS_TABLE = alltables[['ID','ShopName','Rate','Min_wait','Max_wait','Del_status','Del_kind']]
SHOPS_TABLE.head(4)

,ID,ShopName,Rate,Min_wait,Max_wait,Del_status,Del_kind
0,1,کافه رستوران اسکای,4.8,30,50,1,آژانس
1,2,تهیه غذای منظریه,0.0,30,50,1,آژانس
2,3,رستوران ژیوان,0.0,20,40,1,آژانس
3,4,پیتزا تک,0.0,40,60,1,آژانس


## 2- DRIVING LOCATIONS TABLE

In [22]:
LOCATIONS_TABLE = alltables[['ID','City','Lat','Lon','Address']]
LOCATIONS_TABLE.head(4)

,ID,City,Lat,Lon,Address
0,1,اراک,34.08529,49.68840,خیابان عباس آباد، روبه روی پاساژ اسلامی، کوچه ...
1,2,اراک,34.08920,49.68090,اراک ، پایین تر از پل فرنگ، خیابان کاشانی،تهیه...
2,3,اراک,34.07695,49.69774,اراک، خیابان شریعتی، نبش کوچه پامچال
3,4,اراک,34.09574,49.68300,اراک، سه راه ابوذر، خیابان ابوذر، روبروی نانوا...


## 3- DRIVING MENUS TABLE

In [23]:
# now it's time to clean "Menu" column
# each value of "Menu" column is like this
# so it seems that it needs to be a separated table
alltables.loc[:,'Menu'][0]

"[{'name': 'چیز فرایز', 'price': '65,000 تومان', 'primarythings': 'سیب زمینی+ادویه مخصوص+دیپ چدار+بیکن ممتاز+سس باربیکیو', 'discount': '0'}, {'name': 'ماشروم فرایز', 'price': '60,000 تومان', 'primarythings': 'سیب زمینی+ادویه مخصوص+سس قارچ+پیازچه', 'discount': '0'}, {'name': 'قارچ سوخاری', 'price': '65,000 تومان', 'primarythings': 'قارچ سوخاری+سس مخصوص', 'discount': '0'}, {'name': 'سالاد سزار(مرغ گریل)', 'price': '88,000 تومان', 'primarythings': '3تکه فیله گریل+کاهو فرانسه+کاهو پبچ+زیتون سیاه+گوجه چری +نان سیر +سس مخصوص سزار', 'discount': '0'}, {'name': 'سالاد سزار(مرغ سوخاری)', 'price': '95,000 تومان', 'primarythings': '3تکه فیله سوخاری+کاهو فرانسه+کاهو پبچ+زیتون سیاه+گوجه چری +نان سیر +سس مخصوص سزار', 'discount': '0'}, {'name': 'کلاسیک فرایز', 'price': '50,000 تومان', 'primarythings': 'سیب زمینی+ادویه مخصوص', 'discount': '0'}, {'name': 'نان سیر', 'price': '59,000 تومان', 'primarythings': 'پنیر+زیتون+فلفل دلمه+سس سیر مخصوص+خمیر ایتالیایی 28 سانتی متری', 'discount': '0'}, {'name': 'پاست

In [24]:
# cleaning "Menu" column
menu_list=[]
for i in alltables.loc[:,'Menu']:
    menu_list.append(eval(i))
    
menu_corr = {}
menu_corr['id'] = [j+1 for j in range(len(menu_list)-1) for i in range(len(menu_list[j])-1)]
menu_corr['name'] = [menu_list[j][i]['name'] for j in range(len(menu_list)-1) for i in range(len(menu_list[j])-1) ]
menu_corr['price'] = [menu_list[j][i]['price'] for j in range(len(menu_list)-1) for i in range(len(menu_list[j])-1) ]
menu_corr['primarythings'] = [menu_list[j][i]['primarythings']  for j in range(len(menu_list)-1) for i in range(len(menu_list[j])-1)]
menu_corr['discount'] = [menu_list[j][i]['discount'] for j in range(len(menu_list)-1) for i in range(len(menu_list[j])-1)]
df_menu = pd.DataFrame(menu_corr)

price_list_certain =[]
price_list_min =[]
price_list = list(df_menu['price'])
for i in price_list:
    if 'از' in i:
        price_list_certain.append('')
        price_list_min.append(i)
    else:
        price_list_certain.append(i)
        price_list_min.append('')

df_menu['certain_price']  = pd.DataFrame(price_list_certain)   
df_menu['min_price']  = pd.DataFrame(price_list_min) 
df_menu.drop("price",axis=1, inplace=True)
df_menu['certain_price']= df_menu['certain_price'].str.replace(' تومان', '')
df_menu['min_price']= df_menu['min_price'].str.replace('قیمت از', '')
df_menu['min_price']= df_menu['min_price'].str.replace('تومان', '')

# renaming column's names
df_menu.rename(columns={"id": "ID","name": "MenuName","primarythings": "MainIngredients","discount": "Discount","certain_price": "CertainPrice","min_price": "MinPrice"}, inplace=True)

# replacing values for menu columns
df_menu['CertainPrice'] = df_menu['CertainPrice'].str.replace(',', '')
df_menu['CertainPrice'] = df_menu['CertainPrice'].str.replace('.', '')
df_menu['CertainPrice'] = df_menu['CertainPrice'].str.strip()
df_menu['MinPrice'] = df_menu['MinPrice'].str.replace(',', '')
df_menu['MinPrice'] = df_menu['MinPrice'].str.replace('.', '')
df_menu['MinPrice'] = df_menu['MinPrice'].str.strip()

# changing column's type
df_menu['CertainPrice'] = df_menu['CertainPrice'].replace('',np.nan)
df_menu['MinPrice'] = df_menu['MinPrice'].replace('',np.nan)
MENUS_TABLE = df_menu.astype({"CertainPrice":"float","MinPrice":"float"})



C:\Users\ALPHA\AppData\Local\Temp\ipykernel_6644\1761204772.py:37: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_menu['CertainPrice'] = df_menu['CertainPrice'].str.replace('.', '')
C:\Users\ALPHA\AppData\Local\Temp\ipykernel_6644\1761204772.py:40: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_menu['MinPrice'] = df_menu['MinPrice'].str.replace('.', '')


In [25]:
# as we see the "Menu" has changed to a separated table
MENUS_TABLE.head(3)

,ID,MenuName,MainIngredients,Discount,CertainPrice,MinPrice
0,1,چیز فرایز,سیب زمینی+ادویه مخصوص+دیپ چدار+بیکن ممتاز+سس ب...,0,65000.0,NaN
1,1,ماشروم فرایز,سیب زمینی+ادویه مخصوص+سس قارچ+پیازچه,0,60000.0,NaN
2,1,قارچ سوخاری,قارچ سوخاری+سس مخصوص,0,65000.0,NaN


## 4- DRIVING TIMES TABLE

In [26]:
# now it's time to clean "TimeAvail" column
# each value of "TimeAvail" column is like this
# so it seems that it needs to be a separated table
alltables.loc[:,'TimeAvail'][0]

'صبحانه ناهار شام\nشنبه 10:00:00 تا 12:00:00 12:00:00 تا 17:00:00 17:00:00 تا 23:00:00\nیکشنبه 10:00:00 تا 12:00:00 12:00:00 تا 17:00:00 17:00:00 تا 23:00:00\nدوشنبه 10:00:00 تا 12:00:00 12:00:00 تا 17:00:00 17:00:00 تا 23:00:00\nسه شنبه 10:00:00 تا 12:00:00 12:00:00 تا 17:00:00 17:00:00 تا 23:00:00\nچهارشنبه 10:00:00 تا 12:00:00 12:00:00 تا 17:00:00 17:00:00 تا 23:00:00\nپنجشنبه 10:00:00 تا 12:00:00 12:00:00 تا 17:00:00 17:00:00 تا 23:00:00\nجمعه 10:00:00 تا 12:00:00 12:00:00 تا 17:00:00 17:00:00 تا 23:00:00'

In [27]:
# splitting "TimeAvail" column and renaming the new columns
Available_Time = alltables[['TimeAvail']]
Available_Time = Available_Time['TimeAvail'].str.split('\n', expand=True)
Available_Time.columns = ['Meals','Saturday', 'Sunday', 'Monday' , 'Tuesday' , 'Wednesday' , 'Thursday' , 'Friday']

# adding unique column ID
Available_Time['ID'] = [i for i in range(1,Available_Time.shape[0]+1)]
Available_Time = Available_Time.reset_index(drop =True)
Available_Time.head(3)

,Meals,Saturday,Sunday,Monday,Tuesday,Wednesday,Thursday,Friday,ID
0,صبحانه ناهار شام,شنبه 10:00:00 تا 12:00:00 12:00:00 تا 17:00:00...,یکشنبه 10:00:00 تا 12:00:00 12:00:00 تا 17:00:...,دوشنبه 10:00:00 تا 12:00:00 12:00:00 تا 17:00:...,سه شنبه 10:00:00 تا 12:00:00 12:00:00 تا 17:00...,چهارشنبه 10:00:00 تا 12:00:00 12:00:00 تا 17:0...,پنجشنبه 10:00:00 تا 12:00:00 12:00:00 تا 17:00...,جمعه 10:00:00 تا 12:00:00 12:00:00 تا 17:00:00...,1
1,ناهار شام,شنبه 11:00:00 تا 15:00:00 18:30:00 تا 23:00:00,یکشنبه 10:00:00 تا 15:00:00 18:30:00 تا 23:00:00,دوشنبه 11:00:00 تا 15:00:00 18:30:00 تا 23:00:00,سه شنبه 11:00:00 تا 15:00:00 18:30:00 تا 23:00:00,چهارشنبه 11:00:00 تا 15:00:00 18:30:00 تا 23:0...,پنجشنبه 11:00:00 تا 15:00:00 18:30:00 تا 23:00:00,جمعه 11:00:00 تا 15:00:00 18:30:00 تا 23:00:00,2
2,ناهار شام,شنبه 11:30:00 تا 15:00:00 18:30:00 تا 23:00:00,یکشنبه 11:30:00 تا 15:00:00 18:30:00 تا 23:00:00,دوشنبه 11:30:00 تا 15:00:00 18:30:00 تا 23:00:00,سه شنبه 11:30:00 تا 15:00:00 18:30:00 تا 23:00:00,چهارشنبه 11:30:00 تا 15:00:00 18:30:00 تا 23:0...,پنجشنبه 11:30:00 تا 15:00:00 18:30:00 تا 23:00:00,جمعه 11:30:00 تا 15:00:00 18:30:00 تا 23:00:00,3


In [28]:
# unpivoting is sth will help alot!
Available_Time = Available_Time.melt(id_vars=['ID','Meals'], var_name='WeekDay', value_name='Hours')
Available_Time.head(3)

,ID,Meals,WeekDay,Hours
0,1,صبحانه ناهار شام,Saturday,شنبه 10:00:00 تا 12:00:00 12:00:00 تا 17:00:00...
1,2,ناهار شام,Saturday,شنبه 11:00:00 تا 15:00:00 18:30:00 تا 23:00:00
2,3,ناهار شام,Saturday,شنبه 11:30:00 تا 15:00:00 18:30:00 تا 23:00:00


In [29]:
# it seems that the "Hours" column needs to be separated into some columns
lst = ['شنبه', 'یک شنبه', 'دو شنبه', 'سه شنبه', 'چهارشنبه', 'پنج شنبه','جمعه', 'یک','دو', 'سه', 'چهار', 'پنج','تا','-']
for i in lst:
    Available_Time['Hours'] = Available_Time['Hours'].str.replace(i,'')
Available_Time['Hours'] = Available_Time['Hours'].str.strip()
Available_Time.head(3)

,ID,Meals,WeekDay,Hours
0,1,صبحانه ناهار شام,Saturday,10:00:00 12:00:00 12:00:00 17:00:00 17:00:00...
1,2,ناهار شام,Saturday,11:00:00 15:00:00 18:30:00 23:00:00
2,3,ناهار شام,Saturday,11:30:00 15:00:00 18:30:00 23:00:00


In [30]:
'''
we need to split the "Hours" column into 6 columns but it needed to be done in 2 steps 
once for 'صبحانه ناهار شام' and another time for 'ناهار شام' then we append the result tables
'''
print(Available_Time['Meals'].unique())

['صبحانه ناهار شام' 'ناهار شام']


In [31]:
# the first step of splitting "Hours" column
Available_Time_all = Available_Time[Available_Time['Meals'] == 'صبحانه ناهار شام' ]
Available_Time_all[['breakfast_start','n1','breakfast_end','lunch_start','n2','lunch_end','dinner_start','n3','dinner_end']] = Available_Time_all.Hours.str.split(" ", expand = True)
Available_Time_all.drop(columns = ['n1','n2','n3','Hours'], inplace = True)
Available_Time_all.head(4)

C:\Users\ALPHA\AppData\Local\Temp\ipykernel_6644\3177643161.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Available_Time_all[['breakfast_start','n1','breakfast_end','lunch_start','n2','lunch_end','dinner_start','n3','dinner_end']] = Available_Time_all.Hours.str.split(" ", expand = True)
C:\Users\ALPHA\AppData\Local\Temp\ipykernel_6644\3177643161.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Available_Time_all[['breakfast_start','n1','breakfast_end','lunch_start','n2','lunch_end','dinner_start','

,ID,Meals,WeekDay,breakfast_start,breakfast_end,lunch_start,lunch_end,dinner_start,dinner_end
0,1,صبحانه ناهار شام,Saturday,10:00:00,12:00:00,12:00:00,17:00:00,17:00:00,23:00:00
13,14,صبحانه ناهار شام,Saturday,09:00:00,12:00:00,12:00:00,18:00:00,18:01:00,23:30:00
14,15,صبحانه ناهار شام,Saturday,09:00:00,12:00:00,12:00:00,18:00:00,18:01:00,23:30:00
53,54,صبحانه ناهار شام,Saturday,08:30:00,12:00:00,12:00:00,18:00:00,18:00:00,23:59:00


In [32]:
# the second step of splitting "Hours" column
Available_Time_afew = Available_Time[Available_Time['Meals'] == 'ناهار شام' ]
Available_Time_afew[['lunch_start','n2','lunch_end','dinner_start','n3','dinner_end']] = Available_Time_afew.Hours.str.split(" ", expand = True)
Available_Time_afew.drop(columns = ['n2','n3','Hours'], inplace = True)
Available_Time_afew['breakfast_start'] = ''
Available_Time_afew['breakfast_end'] = ''
Available_Time_afew.head()

C:\Users\ALPHA\AppData\Local\Temp\ipykernel_6644\612823797.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Available_Time_afew[['lunch_start','n2','lunch_end','dinner_start','n3','dinner_end']] = Available_Time_afew.Hours.str.split(" ", expand = True)
C:\Users\ALPHA\AppData\Local\Temp\ipykernel_6644\612823797.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Available_Time_afew[['lunch_start','n2','lunch_end','dinner_start','n3','dinner_end']] = Available_Time_afew.Hours.str.split(" ", expand = True)


,ID,Meals,WeekDay,lunch_start,lunch_end,dinner_start,dinner_end,breakfast_start,breakfast_end
1,2,ناهار شام,Saturday,11:00:00,15:00:00,18:30:00,23:00:00,,
2,3,ناهار شام,Saturday,11:30:00,15:00:00,18:30:00,23:00:00,,
3,4,ناهار شام,Saturday,11:00:00,15:30:00,18:30:00,23:30:00,,
4,5,ناهار شام,Saturday,12:00:00,15:00:00,18:45:00,22:15:00,,
5,6,ناهار شام,Saturday,12:15:00,15:00:00,19:15:00,23:00:00,,


In [33]:
# now it's time to concat the resulted tables of previous steps
Available_Time_afew_final = pd.concat([Available_Time_all,Available_Time_afew], ignore_index=True)
Available_Time_def = Available_Time_afew_final
Available_Time_def.head(3)

,ID,Meals,WeekDay,breakfast_start,breakfast_end,lunch_start,lunch_end,dinner_start,dinner_end
0,1,صبحانه ناهار شام,Saturday,10:00:00,12:00:00,12:00:00,17:00:00,17:00:00,23:00:00
1,14,صبحانه ناهار شام,Saturday,09:00:00,12:00:00,12:00:00,18:00:00,18:01:00,23:30:00
2,15,صبحانه ناهار شام,Saturday,09:00:00,12:00:00,12:00:00,18:00:00,18:01:00,23:30:00


In [34]:
# the meals column doesn't look good, it's better to be separated into 3 columns
meals_all = Available_Time_def[Available_Time_def['Meals'] == 'صبحانه ناهار شام' ]
meals_all.Meals.str.split(" ", expand = True).head()

,0,1,2
0,صبحانه,ناهار,شام
1,صبحانه,ناهار,شام
2,صبحانه,ناهار,شام
3,صبحانه,ناهار,شام
4,صبحانه,ناهار,شام


In [35]:
meals_all[['Breakfast','Lunch','Dinner']] = meals_all.Meals.str.split(" ", expand = True)
meals_all.drop(columns = ['Meals'], inplace = True)
meals_all.head()

C:\Users\ALPHA\AppData\Local\Temp\ipykernel_6644\781347904.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meals_all[['Breakfast','Lunch','Dinner']] = meals_all.Meals.str.split(" ", expand = True)
C:\Users\ALPHA\AppData\Local\Temp\ipykernel_6644\781347904.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meals_all[['Breakfast','Lunch','Dinner']] = meals_all.Meals.str.split(" ", expand = True)
C:\Users\ALPHA\AppData\Local\Temp\ipykernel_6644\781347904.py:1: SettingWithCopyWarning: 
A value is trying to

,ID,WeekDay,breakfast_start,breakfast_end,lunch_start,lunch_end,dinner_start,dinner_end,Breakfast,Lunch,Dinner
0,1,Saturday,10:00:00,12:00:00,12:00:00,17:00:00,17:00:00,23:00:00,صبحانه,ناهار,شام
1,14,Saturday,09:00:00,12:00:00,12:00:00,18:00:00,18:01:00,23:30:00,صبحانه,ناهار,شام
2,15,Saturday,09:00:00,12:00:00,12:00:00,18:00:00,18:01:00,23:30:00,صبحانه,ناهار,شام
3,54,Saturday,08:30:00,12:00:00,12:00:00,18:00:00,18:00:00,23:59:00,صبحانه,ناهار,شام
4,55,Saturday,08:00:00,12:00:00,12:00:00,18:00:00,18:00:00,23:59:00,صبحانه,ناهار,شام


In [36]:
meals_afew = Available_Time_def[Available_Time_def['Meals'] == 'ناهار شام' ]
meals_afew.Meals.str.split(" ", expand = True).head()
meals_afew[['Lunch','Dinner']] = meals_afew.Meals.str.split(" ", expand = True)
meals_afew.drop(columns = ['Meals'], inplace = True)
meals_afew['Breakfast'] = ''
meals_afew.head(4)

C:\Users\ALPHA\AppData\Local\Temp\ipykernel_6644\2887102343.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meals_afew[['Lunch','Dinner']] = meals_afew.Meals.str.split(" ", expand = True)
C:\Users\ALPHA\AppData\Local\Temp\ipykernel_6644\2887102343.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meals_afew[['Lunch','Dinner']] = meals_afew.Meals.str.split(" ", expand = True)
C:\Users\ALPHA\AppData\Local\Temp\ipykernel_6644\2887102343.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy

,ID,WeekDay,breakfast_start,breakfast_end,lunch_start,lunch_end,dinner_start,dinner_end,Lunch,Dinner,Breakfast
56,2,Saturday,,,11:00:00,15:00:00,18:30:00,23:00:00,ناهار,شام,
57,3,Saturday,,,11:30:00,15:00:00,18:30:00,23:00:00,ناهار,شام,
58,4,Saturday,,,11:00:00,15:30:00,18:30:00,23:30:00,ناهار,شام,
59,5,Saturday,,,12:00:00,15:00:00,18:45:00,22:15:00,ناهار,شام,


In [37]:
Available_Time_DataFrame = pd.concat([meals_all,meals_afew], ignore_index=True)
Available_Time_DataFrame.head(3)

,ID,WeekDay,breakfast_start,breakfast_end,lunch_start,lunch_end,dinner_start,dinner_end,Breakfast,Lunch,Dinner
0,1,Saturday,10:00:00,12:00:00,12:00:00,17:00:00,17:00:00,23:00:00,صبحانه,ناهار,شام
1,14,Saturday,09:00:00,12:00:00,12:00:00,18:00:00,18:01:00,23:30:00,صبحانه,ناهار,شام
2,15,Saturday,09:00:00,12:00:00,12:00:00,18:00:00,18:01:00,23:30:00,صبحانه,ناهار,شام


In [38]:
Available_Time_DataFrame = Available_Time_DataFrame.replace(['ناهار','صبحانه','شام'] , '1')
Available_Time_DataFrame['Breakfast'] = Available_Time_DataFrame['Breakfast'].replace('','0')
Available_Time_DataFrame.rename(columns={"breakfast_start": "Breakfast_Start","breakfast_end": "Breakfast_End","lunch_start": "Lunch_Start","lunch_end": "Lunch_End","dinner_start": "Dinner_Start","dinner_end": "Dinner_End"}, inplace=True)
Available_Time_DataFrame.head()

,ID,WeekDay,Breakfast_Start,Breakfast_End,Lunch_Start,Lunch_End,Dinner_Start,Dinner_End,Breakfast,Lunch,Dinner
0,1,Saturday,10:00:00,12:00:00,12:00:00,17:00:00,17:00:00,23:00:00,1,1,1
1,14,Saturday,09:00:00,12:00:00,12:00:00,18:00:00,18:01:00,23:30:00,1,1,1
2,15,Saturday,09:00:00,12:00:00,12:00:00,18:00:00,18:01:00,23:30:00,1,1,1
3,54,Saturday,08:30:00,12:00:00,12:00:00,18:00:00,18:00:00,23:59:00,1,1,1
4,55,Saturday,08:00:00,12:00:00,12:00:00,18:00:00,18:00:00,23:59:00,1,1,1


In [39]:
Available_Time_DataFrame = Available_Time_DataFrame.replace(np.nan,'')
TIME_TABLES = Available_Time_DataFrame 

In [40]:
df_menu.to_csv('TIME_TABLES.csv', index = False)
df_menu.to_csv('MENUS_TABLE.csv', index = False)
df_menu.to_csv('LOCATIONS_TABLE.csv', index = False)
df_menu.to_csv('SHOPS_TABLE.csv', index = False)